In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import pickle
import glob

from scipy import stats as sc_stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

2023-05-13 15:26:46.513242: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
import sys
import os
from os.path import dirname
sys.path.append(dirname("../../"))

In [4]:
from src.edl import dense_layers,dense_loss
from src.weibull_edl import loss_and_layers
from src.exp_utils import synthetic

In [5]:
%load_ext autoreload
%autoreload 2

### experiment variable

In [6]:
EXP_NAME = "exp1" #experiment name
N_EPS = 5 #number of eps values to try
N_REGUL = 10 #number of regularisation values to try

In [7]:
base_dir = f"../../exp_results/{EXP_NAME}"

if len(os.listdir(base_dir)) == 0:
    print(f"{base_dir} is empty")
else:    
    print(f"{base_dir} is not empty, Removing files")
    [os.remove(f) for f in glob.glob(base_dir+"/*")]

../../exp_results/exp1 is not empty, Removing files


### Helper funcs

In [8]:
def plot_results(ax, mu, var, model_type, n): 
    ax.plot(x_test,y_test,label="True Test",alpha = 1.0,color="black")
    ax.scatter(x_train,y_train,label="Train")
    ax.plot(x_test.reshape(n), mu, zorder=3, label="Mean Pred",color="red", alpha = 0.5)
    if model_type == "proposed":
        a = 0.5
        b = 1
        var_check = np.sqrt(var)
    else:
        a = 2
        b = 4
        var_check = var
    ax.fill_between(x=x_test.reshape(n),\
                 y1=(mu - a * var_check).reshape(n), \
                 y2=(mu + a * var_check).reshape(n),\
                 label=f"{a} std PI",color="grey",alpha=0.5)
    ax.fill_between(x=x_test.reshape(n),\
                 y1=(mu - b * var_check).reshape(n), \
                 y2=(mu + b * var_check).reshape(n),\
                 label=f"{b} std PI",color="cyan",alpha=0.2)
#     ax.set_ylim(-10,200)
    ax.legend()
    ax.set_title(f"{model_type} Model")

### Experiment for synthetic data

In [9]:
results_dict={}
model_dict={}
data_dict={}

eps_list = np.linspace(0.2,1.5,N_EPS)
for eps in eps_list:
    #a.gen synthetic data
    x_train, y_train = synthetic.gen_data_weibulll(-4, 4, 3400, eps)
    x_test, y_test = synthetic.gen_data_weibulll(-5, 5, 3400, eps, train=False)
    data_dict[eps] = (x_train, y_train, x_test, y_test)
    
    #b.get k from train data
    rv = sc_stats.weibull_min.fit(y_train, floc=0.0)
    k=float(rv[0])
    print (f"For eps={eps}, fitted k ={k}")
    
    #c.plot data and save fig
    fig,ax = plt.subplots(1,1)
    ax.scatter(x_train,y_train,label="Train")
    ax.plot(x_test,y_test,label="Test",alpha = 0.5, color="orange")
    ax.legend()
    ax.set_title(f"$y \sim x^2 + {eps:.3f}*weibull(shape=1.2)$")
    fig.savefig(base_dir+f"/data_gen_eps={eps}.png")
    plt.close()
    
    #d.initialise and run benchmark model
    results_benchmark_eps = {}
    for c_i in np.logspace(-5,0,N_REGUL):
        try:
            print (f"Fitting for c={c_i}")
            print("fitting benchmark")
            #run benchmark model
            mu_i, var_i, y_pred_i, benchmark_model_i = synthetic.results_benchmark_model(c_i,x_train,y_train,x_test)
            gamma, v, alpha, beta = y_pred_i[:,0], y_pred_i[:,1], y_pred_i[:,2], y_pred_i[:,3]
            mse_benchmark_i = mean_squared_error(y_test,mu_i)
            nll_benchmark_i = dense_loss.NIG_NLL(y_test,gamma,v,alpha,beta).numpy()
            results_dict[(eps,c_i,"benchmark")] = (mse_benchmark_i,nll_benchmark_i)
            model_dict[(eps,c_i,"benchmark")] = benchmark_model_i
            
            print("fitting proposed")
            #run proposed model
            mu_prop_i, var_prop_i,\
            y_pred_prop_i, proposed_model_i = synthetic.results_weibull_model(c_i,x_train,y_train,x_test,k,0)
            alpha,beta = y_pred_prop_i[:,0],y_pred_prop_i[:,1]
            mse_proposed_i = mean_squared_error(y_test,mu_prop_i)
            nll_proposed_i = loss_and_layers.weibull_NLL(y_test,alpha,beta, k, reduce=True).numpy()
            results_dict[(eps,c_i,"proposed")] = (mse_proposed_i,nll_proposed_i)
            model_dict[(eps,c_i,"proposed")] = proposed_model_i
            
            #plot the results
            fig,ax = plt.subplots(1,2,figsize=(12,6))
            plot_results(ax[0],mu_i,var_i,"benchmark",n=3400)
            plot_results(ax[1],mu_prop_i,var_prop_i,"proposed",n=3400)
            fig.suptitle(f"eps={eps}, c= {c_i}")
            fig.savefig(base_dir+f"/results_eps={eps},c={c_i}.png")
            plt.close()
        except Exception as e:
            print (f"Error for eps={eps}, c= {c_i}")
            print (e)
        print()

For eps=0.2, fitted k =0.9779374035334656
Fitting for c=1e-05
fitting benchmark


2023-05-13 15:26:52.856157: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=3.5938136638046256e-05
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0001291549665014884
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0004641588833612782
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0016681005372000592
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.005994842503189409
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.021544346900318846
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.07742636826811278
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.2782559402207126
fitting 

/Users/ashish1610dhiman/data_projects/iuq_project/notebooks/neurips_experiments/../../src/exp_utils/synthetic.py:47: RuntimeWarning: divide by zero encountered in divide
  var = np.sqrt(beta / (v * (alpha - 1)))


fitting proposed
107/107 [==============================] - 0s 1ms/step

For eps=0.525, fitted k =1.102811514284493
Fitting for c=1e-05
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=3.5938136638046256e-05
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0001291549665014884
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0004641588833612782
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0016681005372000592
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.005994842503189409
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.021544346900318846
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for 

...

fitting proposed
107/107 [==============================] - 0s 1ms/step

For eps=1.5, fitted k =1.3415029564810828
Fitting for c=1e-05
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=3.5938136638046256e-05
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0001291549665014884
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0004641588833612782
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.0016681005372000592
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=0.005994842503189409
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 999us/step

Fitting for c=0.021544346900318846
fitting benchmark
fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for

/Users/ashish1610dhiman/data_projects/iuq_project/notebooks/neurips_experiments/../../src/exp_utils/synthetic.py:47: RuntimeWarning: divide by zero encountered in divide
  var = np.sqrt(beta / (v * (alpha - 1)))


fitting proposed
107/107 [==============================] - 0s 1ms/step

Fitting for c=1.0
fitting benchmark


/Users/ashish1610dhiman/data_projects/iuq_project/notebooks/neurips_experiments/../../src/exp_utils/synthetic.py:47: RuntimeWarning: divide by zero encountered in divide
  var = np.sqrt(beta / (v * (alpha - 1)))


fitting proposed
107/107 [==============================] - 0s 1ms/step



### Save results

In [10]:
result_df = pd.DataFrame.from_dict(results_dict,orient="index")
result_df[['eps','c',"model_type"]] = pd.DataFrame(result_df.index.tolist(), index= result_df.index)
result_df = result_df.rename(columns={0:"mse",1:"NLL"})
result_df.to_csv(base_dir+"/mse_nll_results.csv")

In [11]:
exp_metadata = {"data":data_dict, "models": model_dict}

with open(base_dir + f'/exp_metadata.pickle', 'wb') as handle:
    pickle.dump(exp_metadata, handle)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_normal_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    

metadata.json                                  2023-05-13 16:31:42           64
variables.h5                                   2023-05-13 16:31:42      1164768
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_weibull_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
.........19
.........2
.........20
.........21
.........22
.........23
.........24
.........3
.........4
.........5
.........6
.........7
.........8
.........9
..

File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:43         3351
metadata.json                                  2023-05-13 16:31:43           64
variables.h5                                   2023-05-13 16:31:43      1160312
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_normal_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
...

File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:43         3382
metadata.json                                  2023-05-13 16:31:43           64
variables.h5                                   2023-05-13 16:31:43      1164768
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_weibull_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
..

File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:44         3358
metadata.json                                  2023-05-13 16:31:44           64
variables.h5                                   2023-05-13 16:31:44      1160312
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_normal_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
...

...

Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:47         3358
metadata.json                                  2023-05-13 16:31:47           64
variables.h5                                   2023-05-13 16:31:47      1160312
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_normal_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16

File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:47         3382
metadata.json                                  2023-05-13 16:31:47           64
variables.h5                                   2023-05-13 16:31:47      1164768
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_weibull_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
..

Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:48         3358
metadata.json                                  2023-05-13 16:31:48           64
variables.h5                                   2023-05-13 16:31:48      1160312
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_normal_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16

File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:48         3382
metadata.json                                  2023-05-13 16:31:48           64
variables.h5                                   2023-05-13 16:31:48      1164768
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_weibull_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
..

File Name                                             Modified             Size
config.json                                    2023-05-13 16:31:48         3358
metadata.json                                  2023-05-13 16:31:48           64
variables.h5                                   2023-05-13 16:31:48      1160312
Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......dense
.........vars
............0
............1
......dense_1
.........vars
............0
............1
......dense_2
.........vars
............0
............1
......dense_3
.........vars
............0
............1
......dense_4
.........vars
............0
............1
......dense_normal_gamma
.........dense
............vars
...............0
...............1
.........vars
...metrics
......mean
.........vars
............0
............1
...optimizer
......vars
.........0
.........1
.........10
.........11
.........12
.........13
.........14
.........15
.........16
.........17
.........18
...

### Junk below

In [22]:
benchmark = result_df[result_df.model_type=="benchmark"]
benchmark["rnk_"] = benchmark.groupby(["eps","model_type"])["mse"].rank()
best_benchmark = benchmark[benchmark["rnk_"]==1]

proposed = result_df[result_df.model_type=="proposed"]
proposed["rnk_"] = proposed.groupby(["eps","model_type"])["mse"].rank()
best_proposed = proposed[proposed["rnk_"]==1]

/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_39707/3421035992.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  benchmark["rnk_"] = benchmark.groupby(["eps","model_type"])["mse"].rank()
/var/folders/kb/v0p0ypbd0wx1q2qb1kpxmmcc0000gn/T/ipykernel_39707/3421035992.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  proposed["rnk_"] = proposed.groupby(["eps","model_type"])["mse"].rank()


In [24]:
best_results = best_benchmark.merge(best_proposed,on="eps",suffixes=("_bench","_prop"))
best_results

,mse_bench,NLL_bench,eps,c_bench,model_type_bench,rnk__bench,mse_prop,NLL_prop,c_prop,model_type_prop,rnk__prop
0,0.064160,34.440475,0.200,0.000010,benchmark,1.0,0.540929,7.258500,0.000129,proposed,1.0
1,0.154957,12.965533,0.525,0.021544,benchmark,1.0,7.373667,4.985559,0.077426,proposed,1.0
2,0.388431,16.832792,0.850,0.000129,benchmark,1.0,20.237682,4.016967,0.000010,proposed,1.0
3,0.619200,7.289849,1.175,0.077426,benchmark,1.0,28.179178,3.539986,0.000129,proposed,1.0
4,1.208574,6.286582,1.500,0.278256,benchmark,1.0,34.659946,3.125432,0.000010,proposed,1.0


In [25]:
proposed

,mse,NLL,eps,c,model_type,rnk_
"(0.2, 1e-05, proposed)",0.622599,7.381303,0.200,0.000010,proposed,5.0
"(0.2, 3.5938136638046256e-05, proposed)",0.543777,7.310148,0.200,0.000036,proposed,2.0
"(0.2, 0.0001291549665014884, proposed)",0.540929,7.258500,0.200,0.000129,proposed,1.0
"(0.2, 0.0004641588833612782, proposed)",1.283657,7.261554,0.200,0.000464,proposed,9.0
"(0.2, 0.0016681005372000592, proposed)",0.655111,7.262743,0.200,0.001668,proposed,6.0
"(0.2, 0.005994842503189409, proposed)",1.496283,7.390838,0.200,0.005995,proposed,10.0
"(0.2, 0.021544346900318846, proposed)",0.577720,7.344656,0.200,0.021544,proposed,3.0
"(0.2, 0.07742636826811278, proposed)",0.579189,7.265340,0.200,0.077426,proposed,4.0
"(0.2, 0.2782559402207126, proposed)",0.810013,7.364060,0.200,0.278256,proposed,7.0
"(0.2, 1.0, proposed)",1.069617,7.439903,0.200,1.000000,proposed,8.0


In [21]:
benchmark[benchmark["rnk_"]==1]

,mse,NLL,eps,c,model_type,rnk_
"(0.2, 1e-05, benchmark)",0.064160,34.440475,0.200,0.000010,benchmark,1.0
"(0.525, 0.021544346900318846, benchmark)",0.154957,12.965533,0.525,0.021544,benchmark,1.0
"(0.8500000000000001, 0.0001291549665014884, benchmark)",0.388431,16.832792,0.850,0.000129,benchmark,1.0
"(1.175, 0.07742636826811278, benchmark)",0.619200,7.289849,1.175,0.077426,benchmark,1.0
"(1.5, 0.2782559402207126, benchmark)",1.208574,6.286582,1.500,0.278256,benchmark,1.0


In [12]:
# mu_prop_i, var_prop_i,\
#             y_pred_prop_i, proposed_model_i = synthetic.results_weibull_model(c_i,x_train,y_train,x_test,k,1)

In [13]:
# synthetic.results_weibull_model(c_i,x_train,y_train,x_test,k,1)

from scipy.special import loggamma

def my_func(x):
    a = loggamma(x-(2/1.05))
    b = loggamma(x)
    return a/b

x = np.linspace(2,40,100)
plt.plot(x,my_func(x))